#IMPORTAÇÕES

In [ ]:
import time
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from PySide6.QtWidgets import (QApplication, QMainWindow, QWidget, QHBoxLayout, QVBoxLayout, QGridLayout,
                               QPushButton, QLabel, QGroupBox, QComboBox, QDialog, QScrollArea, QErrorMessage,
                               QMessageBox, QCheckBox, QLineEdit, QFileDialog)
from PySide6.QtCore import Qt, QThread, Signal, QFileInfo
from PySide6.QtGui import QPixmap, QFont

In [ ]:
class ThreadStandart(QThread):
    signal_wait = Signal(int)
    def run(self):
        for i in range(101):
            time.sleep(0.01)
            self.signal_wait.emit(i)

#CARREGAMENTO DA INTERFACE

In [ ]:
class IntroductionInterface(QMainWindow):
    def __init__(self):
        super().__init__()
        self.init_ui()

    def init_ui(self):
        # system ===============
        self.setFixedSize(600,600)
        self.setWindowTitle('ML - Regressão Linear')
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.central_layout = QVBoxLayout()
        self.central_widget.setLayout(self.central_layout)
        # =======================

        label_background = QLabel()
        pixmap = QPixmap('img/background_img.jpeg')
        label_background.setPixmap(pixmap)
        layout_content = QVBoxLayout()
        layout_content.setContentsMargins(0,0,0,0)
        label_background.setLayout(layout_content)
        self.central_layout.addWidget(label_background)
        self.central_layout.setContentsMargins(0,0,0,0)

        self.label_porcent = QLabel()
        label_spaces = [QLabel() for i in range(2)]

        label_spaces[0].setFixedHeight(405)
        label_spaces[1].setFixedWidth(450)
        layout_adjust_right = QHBoxLayout()
        self.label_porcent.setFixedHeight(50)
        layout_adjust_right.addWidget(label_spaces[1])
        layout_adjust_right.addWidget(self.label_porcent)

        layout_content.addWidget(label_spaces[0])
        layout_content.addLayout(layout_adjust_right)
        
        self.label_porcent.setText('0%')
        font_model = QFont()
        font_model.setPixelSize(40)
        font_model.setBold(True)
        self.label_porcent.setStyleSheet('color:white')
        self.label_porcent.setFont(font_model)
        self.QWorker = ThreadStandart()
        self.QWorker.start()
        self.QWorker.signal_wait.connect(self.update_porcent)
        

    def update_porcent(self, value):
        converter_to_int = str(value)
        self.label_porcent.setText(f'{converter_to_int}%')
        if int(converter_to_int) >= 100:
            self.interfaceOptions = InterfaceApp()
            self.interfaceOptions.show()
            self.close()

    


#FUNÇÃO DE REGRESSÃO LINEAR

In [ ]:
class LinearRegressionFunction():
    def __init__(self, X=None, Y=None, test_precision=0.2, plot_title='', x_label='', y_label=''):
        self.X = X
        self.Y = Y
        self.test_precision = test_precision
        self.plot_title = plot_title
        self.x_label = x_label
        self.y_label = y_label

    def label_encoder():
        pass
    
    def imputer_function(self,strategy_imputer=None, missing_type=None, imputer_bool=False):
        try:
            if imputer_bool:
                X_verify = self.X.values
                Y_verify = self.Y.values

                if np.isnan(X_verify).any() and not np.isnan(Y_verify).any():
                    X_imputer = SimpleImputer(strategy=strategy_imputer)
                    X_Imputed = X_imputer.fit_transform(self.X)
                    self.train_function(X_Imputed, self.Y)

                elif np.isnan(Y_verify).any() and not np.isnan(X_verify).any():
                    Y_imputer = SimpleImputer(strategy=strategy_imputer)
                    Y_Imputed = Y_imputer.fit_transform(self.Y)
                    self.train_function(self.X,Y_Imputed)

                elif np.isnan(X_verify).any() and np.isnan(Y_verify).any():
                    X_imputer = SimpleImputer(strategy=strategy_imputer)
                    X_Imputed = X_imputer.fit_transform(self.X)

                    Y_imputer = SimpleImputer(strategy=strategy_imputer)
                    Y_Imputed = Y_imputer.fit_transform(self.Y)

                    self.train_function(X_Imputed, Y_Imputed)
            
                else:
                    self.train_function(self.X, self.Y)
            else:
                self.train_function(self.X, self.Y)
        except TypeError as err:
            print(err)


    def train_function(self, X_target, Y_target):
        XTrain, XTest, yTrain, yTest = train_test_split(X_target, Y_target, test_size=self.test_precision)
        regress = LinearRegression()
        regress.fit(XTrain, yTrain)

        plt.scatter(XTest, yTest, color = 'red')
        plt.plot(XTest, regress.predict(XTest), color = 'blue')
        plt.title(self.plot_title)
        plt.xlabel(self.x_label)
        plt.ylabel(self.y_label)
        plt.show()

#APLICAÇÃO COM SUAS JANELAS

In [ ]:
class InterfaceApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.init_ui()
        self.groups_design()
    
    def init_ui(self):
        # system ===============
        self.setFixedSize(600,600)
        self.setWindowTitle('Interface')
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.central_layout = QVBoxLayout()
        self.central_widget.setLayout(self.central_layout)
        self.central_layout.setContentsMargins(0,0,0,0)
        # =======================

    def groups_design(self):
        groups_functions = [QGroupBox() for i in range(4)]
        layouts_groups = [QHBoxLayout(), QVBoxLayout(), QVBoxLayout(), QHBoxLayout()]
        names_groups = [' Carregar ',' Coluna ',' Pré-Processamento ',' Exibir Grágico ']
        for i in range(4):
            groups_functions[i].setLayout(layouts_groups[i])
            groups_functions[i].setTitle(names_groups[i])

        groups_functions[1].setFixedWidth(275)
        groups_functions[2].setFixedWidth(275)

        # layouts and groups =========================

        layout_origin = QGridLayout()
        layout_origin.setContentsMargins(5,10,5,20)
        self.central_layout.addLayout(layout_origin)
        layout_origin.addWidget(groups_functions[0],1,1,1,2)
        layout_origin.addWidget(groups_functions[1],2,1,1,1)
        layout_origin.addWidget(groups_functions[2],2,2,1,1)
        layout_origin.addWidget(groups_functions[3],3,1,1,2)

        # ============================================

        self.buttons_actions = [QPushButton() for i in range(4)]
        list_name_buttons = ['Carregar Dados','Conectar ao Banco de Dados','Selecionar Coluna','Plotar']
        for i in range(4):
            self.buttons_actions[i].setText(list_name_buttons[i])

        # upload group =================================
            
        self.buttons_actions[0].setFixedSize(200,90)
        self.buttons_actions[1].setFixedSize(200,90)
        self.buttons_actions[1].setEnabled(False)

        self.buttons_actions[0].clicked.connect(self.open_file)

        layouts_groups[0].addWidget(self.buttons_actions[0])
        layouts_groups[0].addWidget(self.buttons_actions[1])

        # columns group ============================================

        self.buttons_actions[2].setFixedHeight(90)
        self.buttons_actions[2].clicked.connect(self.open_columns)
        self.buttons_actions[2].setEnabled(False)
        self.input_title_plot = QLineEdit()
        self.input_title_plot.setPlaceholderText('Título do Gráfico')

        layouts_groups[1].addWidget(self.buttons_actions[2])
        layouts_groups[1].addWidget(self.input_title_plot)

        # preprocessing group ======================================

        # Imputar dados (dropdown)
        # Rotular dados (checkbox)
        # Normalizar dados / Scaler (checkbox) 

        self.dropdown_imputer = QComboBox()
        list_imputer = ['Nenhum','Média','Mediana','Mais Frequente','Constante']
        self.dropdown_imputer.addItems(list_imputer)

        check_label_encoder = QCheckBox('Rotular')
        check_normalized_data = QCheckBox('Normalizar')

        label_imputer = QLabel('Imputar')
        label_imputer.setFixedHeight(30)
        label_space_pre = QLabel()
        label_space_pre.setFixedHeight(20)

        layouts_groups[2].addWidget(label_imputer)
        layouts_groups[2].addWidget(self.dropdown_imputer)
        layouts_groups[2].addWidget(label_space_pre)
        layouts_groups[2].addWidget(check_label_encoder)
        layouts_groups[2].addWidget(check_normalized_data)

        # plot group ====================================

        self.buttons_actions[3].setFixedSize(400,90)
        self.buttons_actions[3].setEnabled(True)
        self.buttons_actions[3].clicked.connect(self.show_plot)
        layouts_groups[3].addWidget(self.buttons_actions[3])

    def open_columns(self):
        self.window_column = DialogWindowSelectColumn(self.filename)
        self.window_column.exec()
    
    def verification_vetors(self):
        pass
    
    def open_file(self):
        options = QFileDialog.Option()
        self.filename,_ = QFileDialog.getOpenFileName(self,'Abrir Arquivo','','Planilha (*.xlsx);; CSV (*.csv)',options=options)
        self.verification_file()
        
    def verification_file(self):
        if self.filename:
            self.buttons_actions[2].setEnabled(True)
            name_text_file = QFileInfo(self.filename)
            only_name = name_text_file.fileName()
            self.buttons_actions[0].setText(only_name)
        else:
            self.buttons_actions[2].setEnabled(False)

    def show_plot(self):
        linear_function = LinearRegressionFunction(self.window_column.apply_get()[0],self.window_column.apply_get()[1],0.2,self.input_title_plot.text(),self.window_column.name_columns()[0],self.window_column.name_columns()[1])
        if self.dropdown_imputer.currentIndex() == 0:
            linear_function.imputer_function('mean',None,False)
            print('Não selecionou a estrategia')
        elif self.dropdown_imputer.currentIndex() == 1:
            linear_function.imputer_function('mean',None,True)
        elif self.dropdown_imputer.currentIndex() == 2:
            linear_function.imputer_function('median',None,True)
        elif self.dropdown_imputer.currentIndex() == 3:
            linear_function.imputer_function('most_frequent',None,True)
        else:
            linear_function.imputer_function('constant',None,True)

    

class DialogWindowSelectColumn(QDialog):
    def __init__(self, target_data):
        super().__init__()
        self.target_data = target_data
        self.setFixedSize(1100, 500)
        self.setWindowTitle('Selecione as Colunas')
        self.central_layout = QVBoxLayout()
        self.setLayout(self.central_layout)
        self.setup_areas()

    def setup_areas(self):
        layout_main = QGridLayout()
        self.central_layout.addLayout(layout_main)

        groups_columns = [QGroupBox() for i in range(3)]
        layout_groups = [QHBoxLayout() for i in range(3)]

        names_group = ['Independente','Dependente','Finalizar']
        for i in range(3):
            groups_columns[i].setLayout(layout_groups[i])
            groups_columns[i].setTitle(names_group[i])

        area_column = [QScrollArea() for i in range(2)]
        widget_area = [QWidget(area_column[0]), QWidget(area_column[1])]
        layout_area = [QVBoxLayout(widget_area[0]), QVBoxLayout(widget_area[1])]

        layout_content = [QGridLayout() for i in range(2)]
        for i in layout_content:
            i.setAlignment(Qt.AlignmentFlag.AlignTop)
        
        for i in range(2):
            area_column[i].setWidget(widget_area[i])
            area_column[i].setWidgetResizable(True)

        layout_main.addWidget(groups_columns[0],1,1,1,1)
        layout_main.addWidget(groups_columns[1],1,2,1,1)
        layout_main.addWidget(groups_columns[2],2,1,1,2)

        layout_groups[0].addWidget(area_column[0])
        layout_groups[1].addWidget(area_column[1])

        layout_area[0].addLayout(layout_content[0])
        layout_area[1].addLayout(layout_content[1])

        # pattern =======================

        self.buttons_access_d = []
        self.buttons_access_i = []

        self.column_a = 'Nenhum'
        self.column_b = 'Nenhum'

        self.label_column_a = QLabel('Independente: Nenhum')
        self.label_column_b = QLabel('Dependente: Nenhum')

        self.label_column_a.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.label_column_b.setAlignment(Qt.AlignmentFlag.AlignCenter)

        self.model_grid_area(layout_content[0], self.buttons_access_i)
        self.model_grid_area(layout_content[1], self.buttons_access_d)


        for i in self.buttons_access_i:
            i.clicked.connect(lambda checked=None, text=i.text(): self.select_column(text,self.label_column_a, 0))
            
        for i in self.buttons_access_d:
            i.clicked.connect(lambda checked=None, text=i.text(): self.select_column(text,self.label_column_b, 1))
        # ===============================

        self.button_apply = QPushButton('Aplicar')
        button_clear = QPushButton('Limpar')
        self.button_apply.setEnabled(False)
        self.button_apply.clicked.connect(lambda: self.close())
        button_clear.clicked.connect(lambda: self.reset_column(self.button_apply))

        layout_groups[2].addWidget(self.button_apply)
        layout_groups[2].addWidget(button_clear)
        layout_groups[2].addWidget(self.label_column_a)
        layout_groups[2].addWidget(self.label_column_b)

        

        self.normalized_button()
    
    def reset_column(self, apply_btn:QPushButton):
        self.column_a = 'Nenhum'
        self.column_b = 'Nenhum'

        self.label_column_a.setText(self.column_a)
        self.label_column_b.setText(self.column_b)

        apply_btn.setEnabled(False)

    def normalized_button(self):
        style_not_pressed = '''
                               QPushButton
                               {
                                background-color: #bfbcbb; 
                                color: black; 
                                border: 0px solid black;
                               }

                               QPushButton:hover
                               {
                                background-color: #adacac; 
                                color: black; 
                                border: 0px solid black;
                               }
                            
                               QPushButton:pressed
                               {
                               background-color: blue;
                               color: white; 
                               }
                                '''
        for i in self.buttons_access_i:
            i.setStyleSheet(style_not_pressed)

        for i in self.buttons_access_d:
            i.setStyleSheet(style_not_pressed)

    def select_column(self, target, target_label, index):
        if index == 0:
            target_label.setText(f'Independente: {target}')
            self.column_a = target
            column_print = self.sheets[[target]]
            #print(column_print)
        elif index == 1:
            target_label.setText(f'Dependente: {target}')
            self.column_b = target
            column_print = self.sheets[[target]]
            #print(column_print)
        self.verification_apply()

    def verification_apply(self):
        if self.column_a != 'Nenhum' and self.column_b != 'Nenhum':
            self.button_apply.setEnabled(True)
        else:
            self.button_apply.setEnabled(False)

    def apply_get(self):
        x = self.sheets[[self.column_a]]
        y = self.sheets[[self.column_b]]
        return x,y

    def name_columns(self):
        x = self.column_a
        y = self.column_b
        return x,y

    def model_grid_area(self, target_layout:QGridLayout, button_list:list):
        open_file = None
        if '.xlsx' in self.target_data:
            open_file = pd.read_excel(self.target_data)
        elif '.csv' in self.target_data:
            open_file = pd.read_csv(self.target_data)
        self.sheets = open_file
        headers = open_file.columns.tolist()

        row, column = 1,1
        for i in range(headers.__len__()):
            if row > 2:
                column += 1
                row = 1

            button = QPushButton(f'{headers[i]}')
            button_list.append(button)
            button.setFixedHeight(80)

            target_layout.addWidget(button,column, row,1,1)
            row += 1

#EXECUTAR SCRIPT

In [ ]:
if __name__ == '__main__':
    app = QApplication()
    window = IntroductionInterface()
    window.show()
    sys.exit(app.exec())